In [ ]:
from scipy.io import wavfile
import scipy.io
from scipy import signal
from scipy.fft import fftshift, fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [ ]:
def read_audio(audio) :
    audio_file_path = audio
    samplerate, data = wavfile.read(audio_file_path)
    length = data.shape[0] / samplerate
    return samplerate,data,length

def signal_show(data,samplerate):
    length = data.shape[0] / samplerate
    time = np.linspace(0., length, data.shape[0])
    plt.plot(time, data)

    """plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")"""
    plt.axis('off')
    #plt.show()

def spectrogram(data,size,length,samplerate) :
    NFFT=1024
    #print("Size ",size)
    #print("Length ",length)
    subsampling_rate= size/length
    #print("Subsampling rate: ",subsampling_rate)
    ts = np.arange(size)* (1/subsampling_rate)
    dt=1/subsampling_rate

    window_hann = np.hanning(1024)

    #plt.figure(figsize=(15,8))
    spectrum,freqs,time,im=plt.specgram(data, Fs=subsampling_rate, cmap="jet", NFFT=NFFT
                                        ,noverlap =128,window=window_hann,detrend="mean" )

    spectrum_db = 20 * np.log10(spectrum / np.mean(spectrum))

    """one_d_arr = spectrum_db.flatten()
    sorted_arr = np.sort(one_d_arr)
    percentile_index = int(0.7 * len(sorted_arr))
    threshold = sorted_arr[percentile_index-1]
    spectrum_db[spectrum_db <= threshold] = np.min(spectrum_db)"""


    freq=np.linspace(0,subsampling_rate/2,num=spectrum_db.shape[0])
    #print("Maximum freq: ",np.max(freq))
    df = freq[1]-freq[0]

    #print("spectrum_size",spectrum.shape)
    plt.imshow(
        spectrum_db,
        extent=(ts[0], ts[-1]+dt, freqs[0], freqs[-1]+df),
        origin="lower",
        cmap="jet",
        aspect="auto",vmin=np.min(spectrum_db), vmax=np.max(spectrum_db)
    )
    plt.colorbar()
    #plt.title('Spectrogram ')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    return spectrum_db,np.min(spectrum_db),np.max(spectrum_db)

In [ ]:
def spectrogram2(data,size,length,samplerate) :
    NFFT=1024
    #print("Size ",size)
    #print("Length ",length)
    subsampling_rate= size/length
    #print("Subsampling rate: ",subsampling_rate)
    ts = np.arange(size)* (1/subsampling_rate)
    dt=1/subsampling_rate

    window_hann = np.hanning(1024)
    spectrum,freqs,time,im=plt.specgram(data, Fs=subsampling_rate, cmap="jet", NFFT=NFFT
                                        ,noverlap =128,window=window_hann,detrend="mean" )

    spectrum_db = 20 * np.log10(spectrum / np.mean(spectrum))

    """one_d_arr = spectrum_db.flatten()
    sorted_arr = np.sort(one_d_arr)
    percentile_index = int(0.6 * len(sorted_arr))
    threshold = sorted_arr[percentile_index-1]
    spectrum_db[spectrum_db <= threshold] = np.min(spectrum_db)"""


    freq=np.linspace(0,subsampling_rate/2,num=spectrum_db.shape[0])
    df = freq[1]-freq[0]

    plt.imshow(
        spectrum_db,
        extent=(ts[0], ts[-1]+dt, freqs[0], freqs[-1]+df),
        origin="lower",
        cmap="jet",
        aspect="auto",vmin=np.min(spectrum_db), vmax=np.max(spectrum_db)
    )
    plt.axis('off')
    plt.margins(y=0,x=0)
    #plt.colorbar()
    return spectrum_db

In [ ]:
def spectrogram_afterfilter(data,size,length,samplerate,min_db,max_db) :
    NFFT=1024
    #print("Size ",size)
    #print("Length ",length)
    subsampling_rate= size/length
    print("Subsampling rate: ",subsampling_rate)
    ts = np.arange(size)* (1/subsampling_rate)
    dt=1/subsampling_rate

    window_hann = np.hanning(1024)

    #plt.figure(figsize=(15,8))
    spectrum,freqs,time,im=plt.specgram(data, Fs=subsampling_rate, cmap="jet", NFFT=NFFT
                                        ,noverlap =128,window=window_hann,detrend="mean" )

    #make sure there is no zero in the array
    smallest_positive = np.amin(spectrum[spectrum > 0])
    spectrum=np.where(spectrum <= 0, smallest_positive, spectrum)

    spectrum_db = 20 * np.log10(spectrum / np.mean(spectrum))

    one_d_arr = spectrum_db.flatten()
    sorted_arr = np.sort(one_d_arr)
    percentile_index = int(0.7 * len(sorted_arr))
    threshold = sorted_arr[percentile_index-1]
    spectrum_db[spectrum_db <= threshold] = np.min(spectrum_db)


    freq=np.linspace(0,subsampling_rate/2,num=spectrum_db.shape[0])
    #print("Maximum freq: ",np.max(freq))
    df = freq[1]-freq[0]

    print("spectrum_size",spectrum.shape)
    plt.imshow(
        spectrum_db,
        extent=(ts[0], ts[-1]+dt, freqs[0], freqs[-1]+df),
        origin="lower",
        cmap="jet",
        aspect="auto",vmin=min_db, vmax=max_db
    )
    """plt.colorbar()
    plt.title('Spectrogram of tapir')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')"""

    plt.axis('off')
    plt.margins(y=0,x=0)


    """plotpath='/content/drive/MyDrive/FYP/Spectrogram/Tapir/Tapir_zoo_png/01_01_ch0_zoo.png'
    plt.savefig(plotpath, bbox_inches="tight",pad_inches = 0)"""
    return spectrum_db

In [ ]:
def downsampling(data,samplerate,new_samplerate):
    length = data.shape[0] / samplerate
    new_sample=int(new_samplerate*length)
    new_data = signal.resample(data, new_sample)
    print(new_data.shape)
    return new_data


In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, sr, order):
    nyquist = 0.5 * sr
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def filter_signal(data, sr, lowcut, highcut, order):
    b, a = butter_bandpass(lowcut, highcut, sr, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
def signal_segmentation(data,start,end,samplerate):
    start_point = int(start * samplerate)
    end_point = int(end * samplerate)
    new_data=data[start_point:end_point+1]
    return new_data

# segmentize and save audio (environment noise)

In [ ]:
def write_audio(file_name,data):
    samplerate = 44100
    wavfile.write(file_name, samplerate, data)

In [ ]:
folder_path="D:\FYP dataset new\Animal\Environment noise"
dest_path ="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Environment data\Environment noise_wav"

file_list=os.listdir(folder_path)

for i in range(0, len(file_list)):
    counter = 0
    if file_list[i] ==".ipynb_checkpoints":
        continue
        
    elif(file_list[i][:5] =="shift"):
        continue
    file_path=folder_path+'\\'+file_list[i]
  

    samplerate, data ,length= read_audio(file_path)

    for j in range(int(length)):
        segment_data = signal_segmentation(data[:,0],j,j+1,samplerate)
        counter = j+1
        new_name=dest_path+"\\"+file_list[i][:-4]+"_"+str(counter)+"_.wav"
        write_audio(new_name,segment_data)
    

# Save shift img

In [ ]:
clean_path = "Augmentation sound/Clean/Elephant_clean"

clean_list=os.listdir(clean_path)
shift_list = [file for file in file_list if file[:5] == "shift"]

shift_ori_list=[file[6:] for file in shift_list]


for i in range(0, len(shift_ori_list)):
    ori_img_path = clean_path+"/"+shift_ori_list[i]
    samplerate, data ,length= read_audio(ori_img_path)
    
    plt.figure(figsize=(7, 5))
    spectrogram_value_clean_trim,min_db,max_db=spectrogram(data,data.shape[0],length,samplerate)
    plt.title("Spectrogram clean "+shift_ori_list[i])
    
    dataset_path = folder_path +"/"+"shift_"+shift_ori_list[i]
    samplerate2, data2 ,length2= read_audio(dataset_path)
    
    filter_data=filter_signal(data2,samplerate2,1,15000,2)
    downsample_data= downsampling(filter_data,samplerate, 32000)
    
    plt.figure(figsize=(7, 5))
    spectrogram_value_clean_trim=spectrogram2(downsample_data,downsample_data.shape[0],length2,samplerate2)
    
    new_name="shift_"+ shift_ori_list[i][:-4]+".png"
    plotpath=dest_path + '/'+new_name
  
    print(plotpath)
    #plt.savefig(plotpath, bbox_inches="tight",pad_inches = 0)
        
    

# Save Spectrogram New version


In [ ]:
animal = "Tiger"
folder_path="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Dataset version 1\All spectrogram\\"+ animal
dest_path_train ="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Dataset version 1\\train\\" + animal
dest_path_test ="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Dataset version 1\\test\\" + animal
dest_path_val ="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Dataset version 1\\val\\" + animal
dest_path_rest ="D:\FYP dataset new\MMU folder_FYP_Spectrogram creation\Dataset version 1\\rest\\" + animal





In [ ]:

for i in range(0, len(file_list)):
    counter = 0
    if file_list[i] ==".ipynb_checkpoints":
        continue
        
    """elif(file_list[i][:5] =="shift"):
        continue"""
    file_path=folder_path+'//'+file_list[i]
  

    samplerate, data ,length= read_audio(file_path)

        
    #plt.figure(figsize=(7, 5))
    #spectrogram_value_clean_trim,min_db,max_db=spectrogram(segment_data,segment_data.shape[0],1,samplerate)
    #plt.title("Spectrogram bandpass filter  "+file_list[i])

    filter_data=filter_signal(data,samplerate,1,15000,2)
    downsample_data= downsampling(filter_data,samplerate, 32000)

    plt.figure(figsize=(7, 5))
    spectrogram_value_clean=spectrogram2(downsample_data,downsample_data.shape[0],1,samplerate)

    new_name=file_list[i][:-4]+"_.png"
    plotpath=dest_path_train + '\\'+new_name

    #print(plotpath)
    #spectrogram_list.append(new_name)
    plt.savefig(plotpath, bbox_inches="tight",pad_inches = 0)

    

In [ ]:
import random
from PIL import Image
import os

In [ ]:
spectrogram_list=os.listdir(folder_path)
train_data=random.sample(spectrogram_list, 120)
print(len(spectrogram_list))

for image in train_data:
    file = folder_path+"\\"+image
    img=Image.open(file)
    save_location=dest_path_train+"\\"+image
    img.save(save_location)
    
    

In [ ]:
rest_data_1 = [item for item in spectrogram_list if item not in train_data]
val_data=random.sample(rest_data_1, 40)
print(len(rest_data_1))

for image in val_data:
    file = folder_path+"\\"+image
    img=Image.open(file)
    save_location=dest_path_val+"\\"+image
    img.save(save_location)
    

In [ ]:
rest_data_2 = [item for item in rest_data_1 if item not in val_data]
test_data=random.sample(rest_data_2, 40)
print(len(rest_data_2))

for image in test_data:
    file = folder_path+"\\"+image
    img=Image.open(file)
    save_location=dest_path_test+"\\"+image
    img.save(save_location)

In [ ]:
rest_data_3 = [item for item in rest_data_2 if item not in test_data]
print(len(rest_data_3))

for image in rest_data_3:
    file = folder_path+"\\"+image
    img=Image.open(file)
    save_location=dest_path_rest+"\\"+image
    img.save(save_location)